In [26]:
# get the scripts from github
!git clone https://github.com/aashu-0/FineTuning_GPT2.git
%cd FineTuning_GPT2

Cloning into 'FineTuning_GPT2'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 256 (delta 143), reused 177 (delta 74), pack-reused 0 (from 0)
Receiving objects: 100% (256/256), 1009.80 KiB | 16.03 MiB/s, done.
Resolving deltas: 100% (143/143), done.
/kaggle/working/FineTuning_GPT2/FineTuning_GPT2


In [27]:
# cwd
import os
print(os.getcwd())

/kaggle/working/FineTuning_GPT2/FineTuning_GPT2


In [28]:
!pip install tiktoken

In [29]:
# import

from fine_tune.train import train_model_with_samples, calc_loss_batch, calc_loss_loader, evaluate_model
from fine_tune.dataset import download_dataset, load_subset, train_test_split, create_dataloader
from fine_tune.dataset import InstructionDataset
from base_model.utils import text_to_token_ids, token_ids_to_text, generate
from fine_tune.utils import plot_losses, format_input
from base_model.config import GPT2Config
from base_model.model import GPTModel
from fine_tune.config import TrainingConfig
import time
import tiktoken
import torch
import random

In [54]:
# create dataset and dataloader
config = TrainingConfig()
full_dataset = download_dataset(config)

example = full_dataset[random.randint(0,len(full_dataset))]
formatted_ex = format_input(example)

print(f'Random Example:\n{formatted_ex}\n')

Data Loaded Successfully
Number of entries in dataset: 51760
Random Example:
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
Suggest a recipe that uses only ten ingredients.

### Response:




In [31]:
# train test split
train_data, test_data, val_data = train_test_split(
    full_dataset,config)

Train set size: 43996
Test set size: 2588
Validation set size: 5176


In [32]:
# dataloaders
tokenizer = tiktoken.get_encoding('gpt2')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_loader, test_loader, val_loader = create_dataloader(
    train_data, test_data, val_data,
    tokenizer, config, device=device)

for idx, (X, y) in enumerate(train_loader):
    print(f'Input Shape: {X.shape} | Target Shape: {y.shape}')
    break

Input Shape: torch.Size([8, 365]) | Target Shape: torch.Size([8, 365])


In [33]:
from base_model.load_weights import load_gpt2_weights_to_model
config = GPT2Config()

# load model with pretrained weights
model= load_gpt2_weights_to_model(config)
model.eval();

Loading pre-trained GPT-2 model from Hugging Face...
Initializing custom model...
Weights successfully loaded into custom model.


In [34]:
model

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (trf_blocks): ModuleList(
    (0-11): 12 x Block(
      (ln1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): CausalMultiHeadAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (res_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (gelu): GELU(approximate='tanh')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (final_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (out_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [35]:
# Implementing a LORA layer
import torch.nn as nn
import math

class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        self.A = nn.Parameter(torch.empty(in_dim, rank))
        torch.nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
        self.B = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha
        
    def forward(self,x):
        x = self.alpha*(x @ self.A @ self.B)
        return x

In [36]:
# LinearWithLORA Layer
class LinearWithLORA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features,
            linear.out_features,
            rank,
            alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

In [37]:
# replace linear with linearwithlora layer
def replace_linear_with_lora(model, rank, alpha):
    for name, module in model.named_children():
        if isinstance(module, torch.nn.Linear):
            setattr(model,
                    name,
                    LinearWithLORA(module, rank, alpha)
            )
        else:
            replace_linear_with_lora(module, rank, alpha)

In [38]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Total number of trainable parameters: {total_params:,}')

Total number of trainable parameters: 163,037,184


In [39]:
# frezzing the original parameters
for param in model.parameters():
  param.requires_grad = False

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Now, Total number of trainable parameters: {total_params}')

Now, Total number of trainable parameters: 0


In [40]:
replace_linear_with_lora(model, rank=16, alpha=16)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of LORA trainable parameters: {total_params:,}')

Total number of LORA trainable parameters: 3,175,696


In [41]:
# 163037184//6351392  -> 25x reduction in parameters

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [43]:
model.to(device)

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (trf_blocks): ModuleList(
    (0-11): 12 x Block(
      (ln1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): CausalMultiHeadAttention(
        (qkv): LinearWithLORA(
          (linear): Linear(in_features=768, out_features=2304, bias=True)
          (lora): LoRALayer()
        )
        (out_proj): LinearWithLORA(
          (linear): Linear(in_features=768, out_features=768, bias=True)
          (lora): LoRALayer()
        )
        (res_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (fc1): LinearWithLORA(
          (linear): Linear(in_features=768, out_features=3072, bias=True)
          (lora): LoRALayer()
        )
        (gelu): GELU(approximate='tanh')
        (fc2): LinearWithLORA(
          (linear): Linear(in_features=3072, out_features=

In [55]:
formatted_ex

'Below is an instruction that describes a task.\n\nWrite a response that appropriately completes the request.\n\n### Instruction:\nSuggest a recipe that uses only ten ingredients.\n\n### Response:\n'

In [56]:
# access the raw model(no fine tuning yet)
torch.manual_seed(123)
input_ids = text_to_token_ids(
    formatted_ex, tokenizer).to(device)

out = model(input_ids)
# print(out)
print(out.size())

torch.Size([1, 41, 50257])


In [58]:
out_ids = generate(model=model,
                   idx=input_ids,
                   max_new_tokens=100,
                   context_size= config.context_length,
                   eos_id= 50256,
                   temp=0.9,
                   top_k=100)
text = token_ids_to_text(out_ids, tokenizer)
print(text)

Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
Suggest a recipe that uses only ten ingredients.

### Response:

Please include an information about the food at review.

### Recipe:

Place the ingredients into a container placed at the center of the room.

#### Instructions:

Place a single, small spoonful of salted butter between two plastic-wrapped glass plates. Place a small quantity of borax and the rest in the mouth of a small bowl or jar. Shake the food to break up the rind and to make room for the food to fall out for


In [61]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: Currently logged in as: aashu-0 (aashu-0-mnit). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [62]:
import wandb
wandb.login()

True

In [64]:
from fine_tune.config import TrainingConfig
import time
from transformers import get_scheduler

train_config = TrainingConfig(
    num_epochs =10,
    grad_accum_steps=8,
    batch_size=16,
    wandb_project = 'lora_fine_tuning_alpaca')

model.to(device)
start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=0.00005,
    weight_decay=0.1)

num_training_steps = train_config.num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps
)

train_losses, val_losses, track_tokens_seen = train_model_with_samples(
    model,
    train_loader,
    val_loader,
    optimizer,
    start_context= format_input(val_data[0]),
    lr_scheduler= lr_scheduler,
    device= device,
    tokenizer= tokenizer,
    config= train_config
)

# Save the fine-tuned model
torch.save(model.state_dict(), "gpt2_lorafinetuned.pt")
print("Fine-tuned model saved to 'gpt2_lorafinetuned.pt'")

end_time = time.time()

time_in_mins = (end_time - start_time)/60
print(f'Time taken to train: {time_in_mins:.2f} minutes')

Training model for 10 epochs with gradient accumlation every 8 steps

Epoch 1/10
Epoch: 1/10 | Step: 000005 | Tokens: 136,192
Train Loss: 3.1392 | Val Loss: 3.0192
Epoch: 1/10 | Step: 000010 | Tokens: 248,576
Train Loss: 2.8057 | Val Loss: 3.0165
Epoch: 1/10 | Step: 000015 | Tokens: 367,104
Train Loss: 2.9784 | Val Loss: 3.0117
Epoch: 1/10 | Step: 000020 | Tokens: 508,416
Train Loss: 2.8817 | Val Loss: 3.0048
Epoch: 1/10 | Step: 000025 | Tokens: 650,112
Train Loss: 2.9364 | Val Loss: 2.9956
Epoch: 1/10 | Step: 000030 | Tokens: 767,168
Train Loss: 2.9949 | Val Loss: 2.9840
Epoch: 1/10 | Step: 000035 | Tokens: 892,352
Train Loss: 2.8375 | Val Loss: 2.9696
Epoch: 1/10 | Step: 000040 | Tokens: 1,029,888
Train Loss: 2.8635 | Val Loss: 2.9524
Epoch: 1/10 | Step: 000045 | Tokens: 1,179,456
Train Loss: 2.8455 | Val Loss: 2.9317
Epoch: 1/10 | Step: 000050 | Tokens: 1,307,648
Train Loss: 2.8205 | Val Loss: 2.9068
Epoch: 1/10 | Step: 000055 | Tokens: 1,436,032
Train Loss: 2.7511 | Val Loss: 2.877

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇██
global_step,▁▁▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▇▇▇▇▇▇▇███
tokens_seen,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▇▇▇▇█████
train_loss,█▄▃▃▂▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▂▂▂▂▂▁▁
val_loss,██▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,10
global_step,6880
tokens_seen,179977280
train_loss,1.43963
val_loss,1.75493


Fine-tuned model saved to 'gpt2_lorafinetuned.pt'
Time taken to train: 364.03 minutes


## **Evaluation**

In [77]:
torch.manual_seed(1234)

for entry in test_data[-5:-1]:
    input_text = format_input(entry)
    token_ids = generate(
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=100,
        context_size=config.context_length,
        eos_id=50256,
        temp=0.7,
        top_k=50
    )

    generated_text = token_ids_to_text(token_ids, tokenizer)
    response_text = (generated_text[len(input_text):]
                     .replace("### Response:", "")
                     .strip())

    print(input_text)
    print(f"\nCorrect response:\n> {entry['output']}")
    print(f"\nModel response:\n> {response_text.strip()}")
    print("----------------------------------------")

Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
You will be given a piece of text about an event. Based on the information in the text, you need to answer whether the event is possible or not. Your answer must be either Yes or No. If it is not possible for the event to happen based on the information in the text, then your answer should be 'No'. If it is possible for the event to happen, even if it may not necessarily happen, then your output should be 'Yes'.

### Input:
Text: John went out for a walk with his dog Rover.

### Response:


Correct response:
> Yes

Model response:
> Yes. John went out for a walk with his dog Rover.<|endoftext|>
----------------------------------------
Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
You

### **Generating test set responses and adding them to a dictionary**

In [78]:
# generating test set responses

from tqdm import tqdm
import json  # Make sure this is imported

for i, entry in tqdm(enumerate(test_data), total=len(test_data)):
    input_text = format_input(entry)
    token_ids = generate(
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=100,
        context_size=config.context_length,
        eos_id=50256,
        temp=0.7,
        top_k=50
    )

    generated_text = token_ids_to_text(token_ids, tokenizer)
    response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
    )
    test_data[i]["model_response"] = response_text

with open('instruction_evaluated_data.json', 'w') as file:
    json.dump(test_data, file, indent=4)


100%|██████████| 2588/2588 [36:41<00:00,  1.18it/s]


In [82]:
import shutil

shutil.copy(
    "/kaggle/working/FineTuning_GPT2/FineTuning_GPT2/gpt2_lorafinetuned.pt",
    "/kaggle/working/gpt2_lorafinetuned.pt"
)

'/kaggle/working/gpt2_lorafinetuned.pt'

In [83]:
from IPython.display import FileLink

FileLink(r'gpt2_lorafinetuned.pt')

/kaggle/working/FineTuning_GPT2/FineTuning_GPT2/gpt2_lorafinetuned.pt